In [1]:
!pip install planarity
import networkx as nx
import planarity

     |████████████████████████████████| 193 kB 5.1 MB/s 
  Created wheel for planarity: filename=planarity-0.4.1-cp37-cp37m-linux_x86_64.whl size=288863 sha256=14fd26200044ac577f23b3860b7189a49632dff0b13c480679d46591d98847f3
  Stored in directory: /root/.cache/pip/wheels/ae/19/92/2d33078fe3b703cf34dcf6a670652c1d6a58c0a2ebf2b3a68d
Successfully built planarity


In [35]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import time

In [33]:
# Network Science
# Correlation matrix as input, graph as output
# MST Minimum Spanning Tree
def MST(corr_mat,draw = False):
  # corr_mat is the correlation matrix as np.array format
  G = nx.from_numpy_matrix(corr_mat)
  MST = nx.minimum_spanning_tree(G,weight='weight')
  # MST is the output minimum spanning tree graph
  if draw == True:
    nx.draw_networkx(MST, pos=None, arrows=None, with_labels=True)
  return MST


# PMFG Planar Maximally Filtered Graph
def PMFG(corr_mat,draw = False):
  G = nx.from_numpy_matrix(corr_mat)

  def sort_graph_edges(G):
    sorted_edges = []
    for source, dest, data in sorted(G.edges(data=True), key=lambda x: x[2]['weight'], reverse = True): 
        sorted_edges.append({'source': source,
                             'dest': dest,
                             'weight': data['weight']})
    return sorted_edges # a list of dict

  PMFG = nx.Graph()
  ne_total = G.number_of_edges()
  nb_nodes = len(G.nodes)
  ne_pmfg = 3*(nb_nodes-2)
  sorted_edges = sort_graph_edges(G)
  t0 = time.time()
  for i, edge in enumerate(sorted_edges):
      PMFG.add_edge(edge['source'], edge['dest'], weight = edge['weight'])
      if not planarity.is_planar(PMFG):
          PMFG.remove_edge(edge['source'], edge['dest'])
      ne = PMFG.number_of_edges()
      print("Generating PMFG... added edges in PMFG %d/%d (%.2f%%) lookup edges in G %d/%d (%.2f%%) Elapsed TIme %.2f [sec]"\
          %(ne, ne_pmfg, (ne/ne_pmfg)*100, i, ne_total, (i+1/ne_total)*100, time.time()-t0), end="\r")
      if ne == ne_pmfg:
          break

  if draw == True:
    nx.draw_networkx(PMFG, pos=None, arrows=None, with_labels=True)
  return PMFG
  